In [14]:
import pyspark.sql as sql
from pyspark.sql import SparkSession
dir = "auto-mpg.csv"

In [15]:
import pandas as pd
pd.read_csv(dir).keys()

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name'],
      dtype='object')

In [16]:
spark = SparkSession.builder.appName("Read CSV").getOrCreate()
df = spark.read.csv(dir, header=True, inferSchema=True)
df.show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|       198|  4341|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|       220|  4354|         9.0|        70|     1|    chevrolet impala|


In [17]:
df.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model year: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- car name: string (nullable = true)



In [18]:
df = df.na.drop()

q1 = df.approxQuantile('horsepower', [0.25], 0.05)[0]
q3 = df.approxQuantile('horsepower', [0.75], 0.05)[0]
iqr = q3 - q1
df = df.filter((df.horsepower >= q1 - 1.5 * iqr) & (df.horsepower <= q3 + 1.5 * iqr))

from pyspark.sql.functions import regexp_replace
df = df.withColumn('origin', regexp_replace('origin', '1', 'USA'))
df = df.withColumn('origin', regexp_replace('origin', '2', 'Europe'))
df = df.withColumn('origin', regexp_replace('origin', '3', 'Asia'))

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin'], outputCol='features')
df = assembler.transform(df)


IllegalArgumentException: requirement failed: Quantile calculation for column horsepower with data type StringType is not supported.